In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

2023-09-18 18:42:26.772445: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-18 18:42:27.249335: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
batch_size = 2048
l_rate = 0.001

In [3]:
@tf.keras.saving.register_keras_serializable()
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(units=128, activation=tf.nn.leaky_relu)
        self.dense2 = tf.keras.layers.Dense(units=1024, activation=tf.nn.leaky_relu)
        self.dense3 = tf.keras.layers.Dense(units=128, activation=tf.nn.leaky_relu)
        self.dense4 = tf.keras.layers.Dense(units=1024, activation=tf.nn.leaky_relu)
        self.dense5 = tf.keras.layers.Dense(units=8)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        output = self.dense5(x)
        return output

In [4]:
def valiAll(index_epoch):
    y_v_p = model(X_v)
    va_mse = tf.reduce_mean(tf.square(y_v_p - y_v))
    va_rmse = tf.sqrt(va_mse)
    va_mae = tf.reduce_mean(tf.abs(y_v_p - y_v))
    va_r2 = 1 - tf.reduce_sum(tf.square(y_v_p - y_v)) / tf.reduce_sum(tf.square(y_v - tf.reduce_mean(y_v)))
    print("mse:{} rmse:{} mae:{} r2:{}".format(va_mse, va_rmse, va_mae, va_r2))

In [5]:
test_dataset = pd.read_csv('testset.csv', encoding='utf-8').sample(frac=1).reset_index(drop=True)
X_v = test_dataset.loc[:,'freq':'L2'].to_numpy(dtype = np.float32)
y_v = test_dataset.loc[:,'S11r':'S41i'].to_numpy(dtype = np.float32)

In [6]:
dataset = pd.read_csv('./20-24Trainset.csv', encoding='utf-8').sample(frac=1).reset_index(drop=True)
X = dataset.loc[:,'freq':'L2'].to_numpy(dtype = np.float32)
y = dataset.loc[:,'S11r':'S41i'].to_numpy(dtype = np.float32)
dataset_train = tf.data.Dataset.from_tensor_slices((X, y))
dataset_train = dataset_train.shuffle(buffer_size=X.shape[0])
dataset_train = dataset_train.batch(batch_size)
dataset_train = dataset_train.prefetch(tf.data.experimental.AUTOTUNE)

2023-09-18 18:42:28.904200: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9604 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2023-09-18 18:42:28.904767: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9621 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


In [7]:
model = MLP()
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate)

In [8]:
for i in range(150):
    for X, y in dataset_train:
        with tf.GradientTape() as tape:
            y_pred = model(X)
            tr_mse = tf.reduce_mean(tf.square(y_pred - y))
        tr_rmse = tf.sqrt(tr_mse)
        tr_mae = tf.reduce_mean(tf.abs(y_pred - y))
        tr_r2 = 1 - tf.reduce_sum(tf.square(y_pred - y)) / tf.reduce_sum(tf.square(y - tf.reduce_mean(y)))
        grads = tape.gradient(tr_mse, model.variables)
        optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
    print("epoch:{}".format(i))
    print("train mse:{} rmse:{} mae:{} r2:{}".format(tr_mse, tr_rmse, tr_mae, tr_r2))
    valiAll(i)

2023-09-18 18:42:29.825857: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55a6eb8d4580 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-18 18:42:29.825877: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2023-09-18 18:42:29.825881: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2023-09-18 18:42:29.829161: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-18 18:42:29.945750: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-09-18 18:42:30.051464: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


epoch:0
train mse:0.04369363561272621 rmse:0.20903022587299347 mae:0.1614728718996048 r2:0.6247859001159668
mse:0.12473075091838837 rmse:0.3531724214553833 mae:0.26291313767433167 r2:-0.021580934524536133
epoch:1
train mse:0.03784184902906418 rmse:0.19452981650829315 mae:0.15185917913913727 r2:0.6752128601074219
mse:0.142878919839859 rmse:0.37799328565597534 mae:0.2784363329410553 r2:-0.17021965980529785
epoch:2
train mse:0.03492240607738495 rmse:0.1868753731250763 mae:0.14420479536056519 r2:0.7003465890884399
mse:0.15907402336597443 rmse:0.39884084463119507 mae:0.28967228531837463 r2:-0.302862286567688
epoch:3
train mse:0.03108307346701622 rmse:0.1763039231300354 mae:0.1339125782251358 r2:0.732069730758667
mse:0.15823212265968323 rmse:0.3977839946746826 mae:0.28635430335998535 r2:-0.2959667444229126
epoch:4
train mse:0.030997613444924355 rmse:0.17606139183044434 mae:0.1319442093372345 r2:0.7338395118713379
mse:0.16155442595481873 rmse:0.4019383490085602 mae:0.2892147898674011 r2:-0.32